In [7]:
# Import packages
import os
import tweepy as tw
import pandas as pd
import numpy as np
import re 

folder_name = 'denominator_data/'
# Create data folder
if not os.path.isdir('./'+folder_name):
    os.mkdir('./'+folder_name)

run = 1

file_name = folder_name+str(run)+"_denom_data.xlsx"

In [9]:
# Create cities list of Italy
df_city = pd.read_excel('useful_datasets/italian_city_dataset.xlsx',engine='openpyxl')
df_city['city'] = df_city['city'].str.lower()
df_city['region'] = df_city['region'].str.lower()
df_city['province'] = df_city['province'].str.lower()

cities                  = list(df_city['city'])
regions_of_cities       = list(df_city['region'])
codes_regions_of_cities = list(df_city['region_code'])
provinces               = list(df_city['province'])
lat_provinces           = list(df_city['lat'])
lon_provinces           = list(df_city['lon'])
car                     = list(df_city['car'])

# Create region list of Italy
df_region = pd.read_excel('useful_datasets/italian_regions_dataset.xlsx',engine='openpyxl',index_col=0,header=[0])
code_regions = list(df_region['region_code'])
regions      = list(df_region['region'])
lat_regions  = list(df_region['lat'])
lon_regions  = list(df_region['lon'])

In [10]:
# Read dataset
data = pd.read_csv('out.csv',header=[0])

# Lower the characheters and remove some words
data["user_location"] = data["user_location"].str.lower()
words = ['italia','italy','','terra','earth']
data = data[~data["user_location"].isin(words)]

# Replace all the special characters with a null string
data["user_location"] = data["user_location"].str.replace(r'[\-\!\@\#\$\%\^\&\*\(\)\_\+\[\]\;\'\.\,\/\{\}\:\"\<\>\?]' , '')

dim = data.shape[0]
print('Dataset dim:',dim)

Dataset dim: 35


In [11]:
# If present, extract region from location information
for i in range(len(regions)):
    mask = data["user_location"].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
    data.loc[mask,"code_region"] = code_regions[i] 
    data.loc[mask,"region"]      = regions[i]   
    data.loc[mask,"lat_region"] = lat_regions[i]
    data.loc[mask,"lon_region"] = lon_regions[i]
    #tweet_data.loc[mask, 'location'] = tweet_data.loc[mask, 'location'].str.replace(reg,'')
    
# If present, extract province from location information
for i in range(len(cities)):
    mask = data["user_location"].str.contains(fr'\b{cities[i]}\b', regex=True, case=False)
    data.loc[mask,"city"]         = cities[i]
    data.loc[mask,"province"]     = provinces[i]
    data.loc[mask,"lat_province"] = lat_provinces[i]
    data.loc[mask,"lon_province"] = lon_provinces[i]
    data.loc[mask,"region"]       = regions_of_cities[i] 
    data.loc[mask,"code_region"]  = codes_regions_of_cities[i]

# If present, extract province information from car
for i in range(len(car)):
    mask = data["user_location"].str.contains(fr'\b{car[i]}\b', regex=True, case=False)
    data.loc[mask,"province"]     = provinces[i]
    data.loc[mask,"lat_province"] = lat_provinces[i]
    data.loc[mask,"lon_province"] = lon_provinces[i]
    data.loc[mask,"region"]       = regions_of_cities[i] 
    data.loc[mask,"code_region"]  = codes_regions_of_cities[i]
    
# Remove row if 'region' & 'city' are null
data = data.drop(data[(data["region"].isnull()) & (data["city"].isnull())].index)

# If present, extract region from location information
for i in range(len(regions)):
    mask = data["region"].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
    data.loc[mask,"lat_region"] = lat_regions[i]
    data.loc[mask,"lon_region"] = lon_regions[i]
    
# Remove city None
data = data[~data["user_location"].isin(['none'])]
print('Number of rows:', data.shape[0],'out of', dim)

In [14]:
# Convert tweet dataset into a excel file
data.to_excel(file_name)  

data.head(5)

,user_id,user_location,tweet_created_at,tweet_id,Unnamed: 4,code_region,region,lat_region,lon_region,city,province,lat_province,lon_province
0,291252019,arezzo,2021-01-06 18:35:56,1346888246144471042,NaN,9.0,toscana,43.769871,11.255576,arezzo,arezzo,43.517202,11.763879
1,2420824336,lombardia italia,2021-01-06 18:36:00,1346888262451912711,NaN,3.0,lombardia,45.466800,9.190500,NaN,NaN,NaN,NaN
2,1250478054251642900,taranto italy,2021-01-06 18:36:02,1346888270353985540,NaN,16.0,puglia,41.125784,16.862029,taranto,taranto,40.548816,17.080580
3,564816340,milano italia sheher,2021-01-06 18:36:08,1346888294366404611,NaN,3.0,lombardia,45.466800,9.190500,milano,milano,45.466800,9.190500
4,1250079154139267074,taranto puglia,2021-01-06 18:36:11,1346888308195024899,NaN,16.0,puglia,41.125784,16.862029,taranto,taranto,40.548816,17.080580
